# 2.2 OpenAI等SDK调用

### 一、OpenAI Chat SDK 核心特性
OpenAI 提供了多种语言的官方及社区维护的 SDK，支持快速集成 ChatGPT 功能到应用程序中，核心特性包括：
1. **多语言支持**：覆盖 Python、Java、.NET 等主流开发语言  
2. **功能完备性**：支持聊天补全、函数调用、流式响应等核心 API 功能  
3. **开发友好性**：提供同步/异步接口、预构建请求对象、自动错误重试等机制

OpenAI 标准 Python SDK 的核心功能与参数设置,兼容主流大模型调用场景（如 ChatGPT、GPT-4 等），且参数逻辑可扩展至其他兼容 OpenAI API 的国产大模型：

---

### 一、SDK 核心功能与兼容性设计
1. **统一接口规范**  
   OpenAI SDK 通过标准化的 `ChatCompletion.create()` 方法提供模型调用，兼容所有遵循 OpenAI 接口规范的服务（如百度文心、智谱清言等国产大模型）。  
   ```python
   from openai import OpenAI
   client = OpenAI(api_key="YOUR_KEY", base_url="https://国产模型API地址")  # 兼容性关键参数
   ```

2. **跨模型参数映射**  
   核心参数（如 `temperature`、`max_tokens`）在不同模型中语义一致，仅需调整数值范围即可适配不同模型特性。

---

### 二、关键参数设置指南
| 参数             | 作用                                                                 | 推荐值域       | 应用场景                     |
|------------------|----------------------------------------------------------------------|----------------|------------------------------|
| **model**        | 指定模型版本（如 `gpt-3.5-turbo`）                                   | 模型支持列表 | 控制模型能力与成本           |
| **temperature**  | 输出随机性：低值更确定，高值更创意                                   | 0.2-1.0        | 事实问答（0.2）、创作（0.8） |
| **max_tokens**   | 生成文本最大长度（含输入+输出）                                      | 50-4096        | 控制响应篇幅与 API 成本      |
| **top_p**        | 核采样阈值：仅从累计概率达标的候选词中采样                           | 0.7-0.95       | 平衡多样性与相关性           |
| **stream**       | 启用流式响应（逐字输出）                                             | True/False     | 提升交互体验                 |
| **presence_penalty** | 抑制重复内容（正值抑制，负值允许）                               | -2.0-2.0       | 避免冗余回答                 |

#### 示例：多模型兼容调用
```python
# 调用 GPT-3.5（官方模型）
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "解释量子纠缠"}],
    temperature=0.5,
    max_tokens=500
)

# 调用通义千问（国产模型）
response = client.chat.completions.create(
    model="qwen-max", 
    messages=[{"role": "user", "content": "写一首七言绝句"}],
    temperature=0.7,
    top_p=0.9
)
```

---

### 三、最佳实践与注意事项
1. **参数调优策略**  
   • 组合使用 `temperature`+`top_p` 实现精确控制（如 `temperature=0.8` + `top_p=0.9` 平衡创意与逻辑）
   • 通过 `max_tokens` 限制输出长度，避免超出模型上下文限制（如 GPT-3.5 的 4096 tokens）

2. **异常处理机制**  
   ```python
   try:
       response = client.chat.completions.create(...)
   except openai.APIError as e:
       print(f"API 错误: {e}")
   except openai.RateLimitError:
       print("请求频率超限，请稍后重试")
   ```


这里我们以火山方舟的sdk申请为例，其他的都类似：
https://console.volcengine.com/ark/

<img src="img/huoshan.png" width="600px" />

In [1]:
!pip install --upgrade "openai>=1.0"

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 13.8 MB/s eta 0:00:00


In [35]:
ARK_API_KEY=""

In [43]:
import os
from openai import OpenAI

# 请确保您已将 API Key 存储在环境变量 ARK_API_KEY 中
# 初始化Openai客户端
client = OpenAI(
    # 此为默认路径，您可根据业务所在地域进行配置
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    api_key=ARK_API_KEY,
)

In [44]:
# Non-streaming:
completion = client.chat.completions.create(
    model="doubao-1-5-pro-32k-250115",
    messages=[
        {"role": "user", "content": "常见的无人机模拟仿真系统都有哪些？"},
    ],
    temperature=0.1,
)
print(completion.choices[0].message.content)

常见的无人机模拟仿真系统可以分为开源和商业两类，以下是一些典型代表：

### 开源模拟仿真系统
- **Gazebo**
    - **简介**：一款功能强大的机器人和无人机仿真软件，它能够提供高保真的物理模拟和丰富的传感器模型。可以模拟不同的环境，如室内、室外、山地等，支持多种机器人平台，包括各种类型的无人机。
    - **应用场景**：常用于学术研究和开发，例如无人机的路径规划算法测试、传感器融合算法验证等。研究人员可以在Gazebo中创建虚拟环境，对无人机的各种行为进行模拟和分析。
 - **FlightGear**
    - **简介**：是一个开源的飞行模拟器，它具有高度可定制性和广泛的飞行器模型库。可以模拟各种飞行条件和环境，包括不同的气象条件、地形地貌等。
    - **应用场景**：既可以用于无人机飞行控制算法的开发和测试，也可以用于飞行培训。开发者可以利用FlightGear的API接口，将自己的控制算法集成到仿真系统中，对无人机的飞行性能进行评估。
 - **JSBSim**
    - **简介**：是一个开源的飞行器动力学建模和仿真工具，它提供了详细的飞行器动力学模型，能够准确模拟无人机的飞行力学特性。
    - **应用场景**：主要用于无人机飞行控制系统的设计和验证。通过JSBSim，可以对无人机的飞行性能进行精确分析，评估不同控制策略的效果。

### 商业模拟仿真系统
- **MATLAB/Simulink**
    - **简介**：MATLAB是一种广泛应用于科学计算和工程领域的软件，Simulink是其可视化建模和仿真工具。它提供了丰富的模块库，包括无人机动力学模型、控制器设计模块等。
    - **应用场景**：在无人机控制系统设计、算法开发和性能分析方面具有广泛的应用。工程师可以使用Simulink搭建无人机的仿真模型，对控制算法进行快速验证和优化。
 - **X-Plane**
    - **简介**：是一款专业的飞行模拟软件，具有高度真实的飞行物理模型和丰富的地形数据。它支持多种类型的飞行器，包括无人机。
    - **应用场景**：常用于飞行培训、航空研究和无人机系统的评估。其真实的飞行模拟环境可以让用户体验到接近真实飞行的感觉，对于无人机的飞行性能测试和飞行员培训具有重要价值。
 - **Air

## role的使用



是OpenAI SDK中消息角色（Role）的对比表格如下，展示了不同角色的核心功能、典型用途及示例：

| 角色 (Role) | 核心功能                                     | 典型用途                                                                 | 示例                                                                                  | 引用  |
|-------------|---------------------------------------------|-------------------------------------------------------------------------|---------------------------------------------------------------------------------------|-------|
| **`system`** | 设定AI的行为模式、响应规则及操作限制         | - 定义AI身份（如命令行专家）<br>- 控制输出格式（JSON/自然语言）<br>- 设置安全权限 | `{"role": "system", "content": "你需以用户视角判断应执行的命令，当前系统是linux"}`       |   |
| **`user`**   | 传递用户的实际请求或对话输入                 | - 单轮直接提问（如“查看文件”）<br>- 多轮追问（如“具体步骤是什么？”）              | `{"role": "user", "content": "帮我看下当前目录下有没有main.go文件?"}`                   |   |
| **`assistant`** | 存储AI的历史回复内容，用于上下文连贯性维护 | - 多轮对话中提供连贯回答<br>- 开发者可修改历史回复以修正错误                        | `{"role": "assistant", "content": "执行命令：`ls main.go`"}`                          |   |

---

### 补充说明：
1. **角色交互规则**  
   • **消息顺序**：最后一条必须为`user`角色以触发响应，且每次请求需包含完整上下文。
   • **动态调整**：通过插入新的`system`消息可切换AI身份（如从翻译助手变为数据分析师）。

2. **性能优化**  
   • `system`内容建议简洁（200 tokens内），避免冗余影响模型处理效率。

3. **扩展应用**  
   • 结合`function calling`功能，可通过`system`设定函数调用规则，`user`触发请求，`assistant`返回结构化参数以执行外部操作（如调用API）。


In [45]:
#系统角色
completion = client.chat.completions.create(
    model="doubao-1-5-pro-32k-250115",
    messages=[
        {"role": "system", "content": "你需以用户视角判断应执行的命令，仅返回最优选项，当前系统是linux"},
        {"role": "user", "content": "如何查看操作系统类型"},
    ]
)
print(completion.choices[0].message.content)

cat /etc/os-release


## 多轮对话

以下是单轮对话与多轮对话的核心区别及特点总结：

---

### **单轮对话**  
**定义**：用户与AI仅进行一次交互，AI仅基于当前输入生成响应，不保留历史对话信息。  
**特点**：  
1. **无上下文依赖**：每次请求独立处理，适合简单问答或一次性任务（如生成文章、翻译短句）。  
2. **实现简单**：无需管理对话历史，API调用仅需包含`system`指令和`user`输入。  
3. **资源消耗低**：适用于短文本（<500 tokens），响应速度快。  

**示例场景**：  
• 生成文章大纲：用户输入主题，AI直接输出完整结构。  
• 翻译单句：用户提供待翻译文本，AI返回目标语言结果。  
**代码示例**（OpenAI）：  
```python
messages = [
    {"role": "system", "content": "你是一个翻译助手"},
    {"role": "user", "content": "将'Hello'翻译为中文"}
]
```

---

### **多轮对话**  
**定义**：用户与AI进行多次交互，AI需维护上下文以实现连贯对话。  
**特点**：  
1. **上下文管理**：通过`messages`数组传递历史对话（包含`user`和`assistant`角色），使AI理解当前对话状态。  
2. **动态调整能力**：可插入新的`system`指令或修改历史消息，灵活控制AI行为。  
3. **复杂场景支持**：适用于需多步骤交互的任务（如客服问答、代码调试）。  

**实现挑战**：  
• **Token限制**：模型对输入长度有上限（如DeepSeek支持128K tokens），需通过截断历史或生成摘要优化。  
• **性能优化**：流式调用（逐块返回）可提升长文本交互体验，但需处理数据拼接逻辑。  

**示例场景**：  
• 客服系统：用户多次追问订单状态，AI基于历史记录逐步解答。  
• 编程助手：用户逐步描述需求，AI生成并修正代码片段。  

**代码示例**（上下文截断策略）：  
```python
# 仅保留最近5组对话以控制Token数
messages = chat_history[-10:]  
```

---

### **技术对比**  
| **维度**       | 单轮对话                     | 多轮对话                      |  
|----------------|-----------------------------|-----------------------------|  
| **上下文依赖** | 无                          | 强依赖历史对话               |  
| **适用场景**   | 简单任务（翻译、摘要）       | 复杂交互（客服、多步骤推理） |  
| **实现复杂度** | 低                          | 高（需管理上下文、Token限制）|  
| **API调用模式**| 同步调用（直接返回完整响应） | 支持流式调用（逐块返回） |  

---

### **选择建议**  
• **优先单轮对话**：任务简单、无需历史参考时（如生成随机菜谱）。  
• **选择多轮对话**：需连贯交互、动态调整AI行为时（如教学辅导、技术排查）。  
• **结合实时API**：使用OpenAI实时API可自动缓存上下文，降低管理复杂度。  



In [46]:
chat_history = []  # 用于保存完整的对话记录

# 第一轮对话
user_input_1 = "世界上最高的山是什么？"
chat_history.append({"role": "user", "content": user_input_1})

response_1 = client.chat.completions.create(
    model="doubao-1-5-pro-32k-250115",
    messages=chat_history
)
chat_history

[{'role': 'user', 'content': '世界上最高的山是什么？'}]

In [47]:
#assistant回复加入记录
ai_response_1 = response_1.choices[0].message.content
chat_history.append({"role": "assistant", "content": ai_response_1})  # 保存AI回复

chat_history

[{'role': 'user', 'content': '世界上最高的山是什么？'},
 {'role': 'assistant',
  'content': '世界上最高的山是珠穆朗玛峰。\n\n它是喜马拉雅山脉的主峰，位于中国与尼泊尔边界上。其岩面高程为8844.43米（2005年测量），雪面高程为8848.86米（2020年测量）。\n\n珠穆朗玛峰山体呈巨型金字塔状，地形极端险峻，环境非常复杂。其攀登难度极大，但也吸引着众多登山爱好者和探险家前往挑战。   '}]

In [48]:
# 第二轮对话（依赖上下文）
user_input_2 = "第二高的呢？"
chat_history.append({"role": "user", "content": user_input_2})

response_2 = client.chat.completions.create(
    model="doubao-1-5-pro-32k-250115",
    messages=chat_history[-10:]  # 截断策略：仅保留最近10条消息控制Token消耗
)

ai_response_2 = response_2.choices[0].message.content

#assistant回复加入记录
chat_history.append({"role": "assistant", "content": ai_response_2})

chat_history

[{'role': 'user', 'content': '世界上最高的山是什么？'},
 {'role': 'assistant',
  'content': '世界上最高的山是珠穆朗玛峰。\n\n它是喜马拉雅山脉的主峰，位于中国与尼泊尔边界上。其岩面高程为8844.43米（2005年测量），雪面高程为8848.86米（2020年测量）。\n\n珠穆朗玛峰山体呈巨型金字塔状，地形极端险峻，环境非常复杂。其攀登难度极大，但也吸引着众多登山爱好者和探险家前往挑战。   '},
 {'role': 'user', 'content': '第二高的呢？'},
 {'role': 'assistant',
  'content': '世界第二高的山是乔戈里峰，它又称K2峰。以下为你介绍它的相关信息：\n### 基本数据\n乔戈里峰的海拔高度为8611米，它是喀喇昆仑山脉的主峰，坐落在喀喇昆仑山的中段。其地理坐标为北纬35°52′55″、东经76°30′51″，位于中国新疆维吾尔自治区塔什库尔干塔吉克自治县与巴基斯坦吉尔吉特-巴尔蒂斯坦交界处。 \n### 攀登难度\n乔戈里峰的攀登难度远远高于珠穆朗玛峰，被称为“野蛮巨峰”。其攀登死亡率极高，在国际登山界有着“没有回报的山峰”之称。截至2023年，仅有约400人成功登顶乔戈里峰，而在攀登过程中遇难的人数却超过了80人。\n### 名称由来\n- “乔戈里”是塔吉克语，意为“高大雄伟”。\n- “K2”中的“K”指喀喇昆仑山，“2”是当时它是喀喇昆仑山脉第二座被考察的山峰。 '}]

In [49]:
# 打印完整对话记录
for msg in chat_history:
    print(f"{msg['role'].capitalize()}: {msg['content']}")

User: 世界上最高的山是什么？
Assistant: 世界上最高的山是珠穆朗玛峰。

它是喜马拉雅山脉的主峰，位于中国与尼泊尔边界上。其岩面高程为8844.43米（2005年测量），雪面高程为8848.86米（2020年测量）。

珠穆朗玛峰山体呈巨型金字塔状，地形极端险峻，环境非常复杂。其攀登难度极大，但也吸引着众多登山爱好者和探险家前往挑战。   
User: 第二高的呢？
Assistant: 世界第二高的山是乔戈里峰，它又称K2峰。以下为你介绍它的相关信息：
### 基本数据
乔戈里峰的海拔高度为8611米，它是喀喇昆仑山脉的主峰，坐落在喀喇昆仑山的中段。其地理坐标为北纬35°52′55″、东经76°30′51″，位于中国新疆维吾尔自治区塔什库尔干塔吉克自治县与巴基斯坦吉尔吉特-巴尔蒂斯坦交界处。 
### 攀登难度
乔戈里峰的攀登难度远远高于珠穆朗玛峰，被称为“野蛮巨峰”。其攀登死亡率极高，在国际登山界有着“没有回报的山峰”之称。截至2023年，仅有约400人成功登顶乔戈里峰，而在攀登过程中遇难的人数却超过了80人。
### 名称由来
- “乔戈里”是塔吉克语，意为“高大雄伟”。
- “K2”中的“K”指喀喇昆仑山，“2”是当时它是喀喇昆仑山脉第二座被考察的山峰。 


## 火山云邀请码

HGE53IOF

DeepSeek满血版免费领啦！邀请好友注册和使用，最高双方可获得145元代金券，免费抵扣3625万tokens，畅享R1与V3模型！参与入口：https://volcengine.com/L/y4kwAjebnK4/  邀请码：HGE53IOF

<img src='img/huoshan2.png' width='640px' />